In [ ]:
import pandas as pd

df = pd.read_csv("../data/processed/enriched_transactions.csv")
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Nodes
pd.concat([
    df[["nameOrig", "sender_name", "sender_email"]].rename(columns={"nameOrig": "user_id", "sender_name": "name", "sender_email": "email"}),
    df[["nameDest", "receiver_name", "receiver_email"]].rename(columns={"nameDest": "user_id", "receiver_name": "name", "receiver_email": "email"})
]).drop_duplicates().to_csv("../data/processed/nodes_person.csv", index=False)

pd.concat([
    df[["sender_phone"]].rename(columns={"sender_phone": "phone"}),
    df[["receiver_phone"]].rename(columns={"receiver_phone": "phone"})
]).drop_duplicates().to_csv("../data/processed/nodes_phone.csv", index=False)

pd.concat([
    df[["sender_ip"]].rename(columns={"sender_ip": "ip"}),
    df[["receiver_ip"]].rename(columns={"receiver_ip": "ip"})
]).drop_duplicates().to_csv("../data/processed/nodes_ip.csv", index=False)

pd.concat([
    df[["sender_company"]].rename(columns={"sender_company": "company"}),
    df[["receiver_company"]].rename(columns={"receiver_company": "company"})
]).drop_duplicates().to_csv("../data/processed/nodes_company.csv", index=False)

# Relationships
df[["nameOrig", "nameDest", "amount", "type", "isFraud"]].rename(columns={
    "nameOrig": "from", "nameDest": "to"
}).to_csv("../data/processed/relationships_sent.csv", index=False)

pd.concat([
    df[["nameOrig", "sender_phone"]].rename(columns={"nameOrig": "user_id", "sender_phone": "phone"}),
    df[["nameDest", "receiver_phone"]].rename(columns={"nameDest": "user_id", "receiver_phone": "phone"})
]).drop_duplicates().to_csv("../data/processed/relationships_uses_phone.csv", index=False)

pd.concat([
    df[["nameOrig", "sender_ip"]].rename(columns={"nameOrig": "user_id", "sender_ip": "ip"}),
    df[["nameDest", "receiver_ip"]].rename(columns={"nameDest": "user_id", "receiver_ip": "ip"})
]).drop_duplicates().to_csv("../data/processed/relationships_logged_from.csv", index=False)

pd.concat([
    df[["nameOrig", "sender_company"]].rename(columns={"nameOrig": "user_id", "sender_company": "company"}),
    df[["nameDest", "receiver_company"]].rename(columns={"nameDest": "user_id", "receiver_company": "company"})
]).drop_duplicates().to_csv("../data/processed/relationships_works_for.csv", index=False)

print("All node and relationship files exported.")


 All node and relationship CSVs exported to 'data/processed/'


C:\Users\gabma\AppData\Local\Temp\ipykernel_19448\130534184.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
